In [1]:
### automatically refresh the buffer

%load_ext autoreload
%autoreload 2

### solve the auto-complete issue

%config Completer.use_jedi = False

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

### lvl 2 setups (systerm)

import os
import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import warnings
warnings.filterwarnings('ignore')
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Wedge, Circle
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import glob
import datetime

In [2]:
gdf = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
input_folder = '/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/'

start_year = 2007
end_year = start_year+3
for year in range(start_year, end_year):  # 1989不包含

    months = range(10, 13) if year == start_year else range(1, 10) if year == (end_year - 1) else range(1, 13)
    # 遍历月份
    for month in months:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        # 获取当前月份的天数
        if month in [1, 3, 5, 7, 8, 10, 12]:
            num_days = 31
        elif month in [4, 6, 9, 11]:
            num_days = 30
        elif month == 2:
            # 考虑闰年
            if (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0):
                num_days = 29  # 闰年
            else:
                num_days = 28  # 平年

        # 遍历每个月的天数
        for day in range(1, num_days + 1):
            
            month_str = f"{month:02}"
            day_str = f"{day:02}"
            input_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}_*.nc'
            ds = xr.open_mfdataset(input_folder + input_file)
            ds_era = xr.Dataset({'p': (['time', 'latitude', 'longitude'], ds.PREC_ACC_NC.values)},
                                coords={'longitude': (['longitude'], ds.XLONG.values[500]),
                                        'latitude': (['latitude'], ds.XLAT.values[:,500]),
                                        'time': ('time', ds.Time.values)})
            ds_era_lon, ds_era_lat = np.meshgrid(ds_era.longitude.values, ds_era.latitude.values, indexing='xy')
            # 转换为 xarray DataArray，确保其维度与 ds_era_clipped 对齐
            ds_era_lon_da = xr.DataArray(ds_era_lon, dims=("latitude", "longitude"), coords={"latitude": ds_era.latitude, "longitude": ds_era.longitude})
            ds_era_lat_da = xr.DataArray(ds_era_lat, dims=("latitude", "longitude"), coords={"latitude": ds_era.latitude, "longitude": ds_era.longitude})

            # 使用 assign_coords 将二维坐标添加到 ds_era_clipped
            ds_era_clipped = ds_era.assign_coords(lon_2d=ds_era_lon_da, lat_2d=ds_era_lat_da)

            lon = ds_era_clipped['lon_2d'].values
            lat = ds_era_clipped['lat_2d'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')

            mask = np.full(ds_era_clipped['p'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds_era_clipped['p'].dims[1:], coords={'latitude': ds_era_clipped['p'].coords['latitude'], 'longitude': ds_era_clipped['p'].coords['longitude']})
            ds_sss = ds_era_clipped.where(mask_da, drop=True)
            ds_sss = ds_sss.drop_vars(['lon_2d', 'lat_2d'])


            original_times = ds_sss.time.values 
            
            lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
            utc_offsets = [-8, -7, -6, -5]
            print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
            for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):
                mask = (ds_sss.longitude >= lon_min) & (ds_sss.longitude < lon_max)
                ds_lon_subset = ds_sss.where(mask, drop=True)
                if ds_lon_subset.latitude.size > 0 and ds_lon_subset.longitude.size > 0:
            
                    adjusted_times = original_times + np.timedelta64(offset, 'h')  # 保持24个时间点
            
                    ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
                    lat_min = ds_lon_subset.latitude.min().values
                    lat_max = ds_lon_subset.latitude.max().values
            
                    lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
                    
                    for i in range(len(lat_splits) - 1):
                        lat_min_split = lat_splits[i]
                        lat_max_split = lat_splits[i + 1]
                        lat_mask = (ds_lon_subset.latitude >= lat_min_split) & (ds_lon_subset.latitude < lat_max_split)
                        ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                        
                        output_folder = '/N/project/Zli_lab/gongg/CONUS404_data/LST/re_UTC/U' + str(offset)+str(i) + '/'
                        output_file = f'PREC_ACC_NC.wrf2d_d01_{year}-{month_str}-{day_str}.nc'
                        output_path = os.path.join(output_folder, output_file)
                        os.makedirs(output_folder, exist_ok=True)
                        ds_lat_subset.to_netcdf(output_folder + output_file)

2025-02-17 01:23:05
2025-02-17 01:23:22
2025-02-17 01:23:38
2025-02-17 01:23:57
2025-02-17 01:24:16
2025-02-17 01:24:36
2025-02-17 01:24:58
2025-02-17 01:25:20
2025-02-17 01:25:39
2025-02-17 01:26:03
2025-02-17 01:26:25
2025-02-17 01:26:48
2025-02-17 01:27:06
2025-02-17 01:27:34
2025-02-17 01:27:52
2025-02-17 01:28:12
2025-02-17 01:28:34
2025-02-17 01:28:57
2025-02-17 01:29:23
2025-02-17 01:29:45
2025-02-17 01:30:03
2025-02-17 01:30:22
2025-02-17 01:30:40
2025-02-17 01:31:02
2025-02-17 01:31:24
2025-02-17 01:31:52
2025-02-17 01:32:10
2025-02-17 01:32:34
2025-02-17 01:33:01
2025-02-17 01:33:30
2025-02-17 01:34:00
2025-02-17 01:34:27
2025-02-17 01:34:29
2025-02-17 01:34:50
2025-02-17 01:35:30
2025-02-17 01:35:58
2025-02-17 01:36:22
2025-02-17 01:36:37
2025-02-17 01:36:56
2025-02-17 01:37:21
2025-02-17 01:37:54
2025-02-17 01:38:25
2025-02-17 01:38:47
2025-02-17 01:39:11
2025-02-17 01:39:34
2025-02-17 01:39:55
2025-02-17 01:40:15
2025-02-17 01:40:34
2025-02-17 01:41:02
2025-02-17 01:41:28


In [2]:
def create_ds(ds, array):
    lonn = ds.lon.values
    latt = ds.lat.values

    ds_ = xr.Dataset({'s': ([ 'lat', 'lon'], array)},
                    coords={'lon': (['lon'], lonn),
                            'lat': (['lat'], latt),})
    return ds_

In [3]:
import numpy as np
import scipy.stats as stats

def calculate_ccscale_slope(temperature_data, precipitation_data):
    # Ensure the data is flat (1D)
    temperature_data = np.ravel(temperature_data)
    precipitation_data = np.ravel(precipitation_data)

    # Early exit if input arrays are empty
    if temperature_data.size == 0 or precipitation_data.size == 0:
        return np.nan, np.nan, np.nan, np.nan

    # Remove data points where either temperature or precipitation contains NaN
    valid_mask = ~np.isnan(temperature_data) & ~np.isnan(precipitation_data)
    temperature_data = temperature_data[valid_mask]
    precipitation_data = precipitation_data[valid_mask]
    
    if temperature_data.size == 0 or precipitation_data.size == 0:
        return np.nan, np.nan, np.nan, np.nan
    
    # Convert Kelvin temperatures to Celsius
    temperature_data_celsius = temperature_data - 273.15

    # Set the temperature bin size and sliding step
    bin_size = 1.0  # Bin size
    step = 0.5      # Sliding step
    min_temp = temperature_data_celsius.min()
    max_temp = temperature_data_celsius.max()
    
    
    # Create sliding temperature bins
    temperature_bins = np.arange(min_temp, max_temp, step)
    overlapping_bins = [(start, start + bin_size) for start in temperature_bins]

    # Collect precipitation data for each temperature bin using a dictionary
    precipitation_per_bin = {i: [] for i in range(len(overlapping_bins))}
    for temp, precip in zip(temperature_data_celsius, precipitation_data):
        for i, (bin_start, bin_end) in enumerate(overlapping_bins):
            if bin_start <= temp < bin_end:
                precipitation_per_bin[i].append(precip)

    # Calculate the 99th percentile of log precipitation, mean temperature, and confidence intervals
    log_precipitation_99 = []
    mean_temperatures = []
    for idx, (bin_start, bin_end) in enumerate(overlapping_bins):
        bin_data = precipitation_per_bin[idx]
        if len(bin_data) >= 80:
            log_precip = np.log(bin_data)
            quantile_99 = np.percentile(log_precip, 99)
            mean_temp = (bin_start + bin_end) / 2
            log_precipitation_99.append(quantile_99)
            mean_temperatures.append(mean_temp)

    # Perform linear regression if we have at least 5 bins with sufficient data
    if len(log_precipitation_99) >= 5:
        slope, intercept, r_value, p_value, std_err = stats.linregress(mean_temperatures, log_precipitation_99)
    else:
        slope = np.nan
        intercept = np.nan

    return mean_temperatures, log_precipitation_99, slope, intercept

# Example usage:
# temperature_data = np.array([...])
# precipitation_data = np.array([...])
# results = calculate_ccscale_slope(temperature_data, precipitation_data)
# print(results)



In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

def calculate_ccscale_quantreg(temperature_data, precipitation_data):
    # 确保数据是平的（一维的）
    temperature_data = np.ravel(temperature_data)
    precipitation_data = np.ravel(precipitation_data)

    # 移除温度或降水量中包含NaN的数据点
    valid_mask = ~np.isnan(temperature_data) & ~np.isnan(precipitation_data)
    temperature_data = temperature_data[valid_mask]
    precipitation_data = precipitation_data[valid_mask]
    # Early exit if input arrays are empty
    if temperature_data.size == 0 or precipitation_data.size == 0:
        return np.nan

    # 将开尔文温度转换为摄氏度
    temperature_data_celsius = temperature_data - 273.15

    # 创建一个 DataFrame 来包含温度和降水量的数据
    data = pd.DataFrame({
        'Temperature': temperature_data_celsius,
        'Precipitation': precipitation_data
    })

    # 定义模型：在 0.99 分位数处进行分位数回归
    quantile_model = sm.QuantReg(np.log(data['Precipitation']), sm.add_constant(data['Temperature']))
    quantile_regression_result = quantile_model.fit(q=0.99).params[1]

    return quantile_regression_result

# 使用实际的 temperature_data 和 precipitation_data 调用该函数
# 例如：
# result = calculate_ccscale_quantreg(temperature_data, precipitation_data)
# print(result.summary())


In [8]:
input_folder_t = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA/'
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/'
file_pattern_p = 'PREC_ACC_NC.wrf2d_d01_????-??-??.nc'

In [ ]:
folder_names = [
    # 'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
     'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
    # 'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
    # 'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]
for folder in folder_names:
    full_path_p = os.path.join(base_path, folder, file_pattern_p)
    all_files_p = glob.glob(full_path_p)
    #####
    summer_files_p = [f for f in all_files_p if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_p = xr.open_mfdataset(summer_files_p)
    ds_p = ds_p.sel(time=ds_p['time'].dt.month.isin([6, 7, 8]))
    ds_t = xr.open_mfdataset(input_folder_t+'dn_temp_'+folder+'.nc')
    ds_p_filtered = ds_p.where(ds_p['p'] > 0.1, np.nan)
    
    # 保留特定时间范围内的数据，其他时间标记为nan
    ds_p_daytime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 6) & (ds_p_filtered['time.hour'] < 18), np.nan)
    # 保留18点到次日早上6点的数据，其他时间标记为nan
    ds_p_nighttime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 18) | (ds_p_filtered['time.hour'] < 6), np.nan)
    
    
    arr_dtp = ds_p_daytime.p.values
    arr_ntp = ds_p_nighttime.p.values
    arr_t = ds_t.dnt.values
    
    arr_dt = np.where(np.isnan(arr_dtp), np.nan, arr_t)
    arr_nt = np.where(np.isnan(arr_ntp), np.nan, arr_t)
    
    arr_slope_nt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    arr_slope_dt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    # 循环遍历每个网格点
    
    for i in range(arr_nt.shape[1]):
        for j in range(arr_nt.shape[2]):
            temperature_data_nt = arr_nt[:, i, j]
            precipitation_data_nt = arr_ntp[:, i, j]
            
            temperature_data_dt = arr_dt[:, i, j]
            precipitation_data_dt = arr_dtp[:, i, j]
            # 调用函数并获取斜率
            slope_nt = calculate_ccscale_slope(temperature_data_nt, precipitation_data_nt)[2]
            slope_dt = calculate_ccscale_slope(temperature_data_dt, precipitation_data_dt)[2]
            # 将斜率值存储到arr_slope的对应位置
            arr_slope_nt[i, j] = slope_nt
            arr_slope_dt[i, j] = slope_dt
            ds_lrs_nt = create_ds(ds_t, arr_slope_nt)
            ds_lrs_dt = create_ds(ds_t, arr_slope_dt)
    ds_lrs_nt.to_netcdf(input_folder_t+'ds_lrs_nt'+folder+'.nc')
    ds_lrs_dt.to_netcdf(input_folder_t+'ds_lrs_dt'+folder+'.nc')

In [ ]:
folder_names = [
    # 'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
    'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
    # 'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
    # 'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]
for folder in folder_names:
    full_path_p = os.path.join(base_path, folder, file_pattern_p)
    all_files_p = glob.glob(full_path_p)
    #####
    summer_files_p = [f for f in all_files_p if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_p = xr.open_mfdataset(summer_files_p)
    ds_p = ds_p.sel(time=ds_p['time'].dt.month.isin([6, 7, 8]))
    ds_t = xr.open_mfdataset(input_folder_t+'dn_temp_'+folder+'.nc')
    ds_p_filtered = ds_p.where(ds_p['p'] > 0.1, np.nan)
    
    # 保留特定时间范围内的数据，其他时间标记为nan
    ds_p_daytime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 6) & (ds_p_filtered['time.hour'] < 18), np.nan)
    # 保留18点到次日早上6点的数据，其他时间标记为nan
    ds_p_nighttime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 18) | (ds_p_filtered['time.hour'] < 6), np.nan)
    
    
    arr_dtp = ds_p_daytime.p.values
    arr_ntp = ds_p_nighttime.p.values
    arr_t = ds_t.dnt.values
    
    arr_dt = np.where(np.isnan(arr_dtp), np.nan, arr_t)
    arr_nt = np.where(np.isnan(arr_ntp), np.nan, arr_t)
    
    arr_slope_nt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    arr_slope_dt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    # 循环遍历每个网格点
    
    for i in range(arr_nt.shape[1]):
        for j in range(arr_nt.shape[2]):
            temperature_data_nt = arr_nt[:, i, j]
            precipitation_data_nt = arr_ntp[:, i, j]
            
            temperature_data_dt = arr_dt[:, i, j]
            precipitation_data_dt = arr_dtp[:, i, j]
            # 调用函数并获取斜率
    
            slope_nt = calculate_ccscale_quantreg(temperature_data_nt, precipitation_data_nt).params[1]
            slope_dt = calculate_ccscale_quantreg(temperature_data_dt, precipitation_data_dt).params[1]
            # 将斜率值存储到arr_slope的对应位置
            arr_slope_nt[i, j] = slope_nt
            arr_slope_dt[i, j] = slope_dt
            ds_qrs_nt = create_ds(ds_t, arr_slope_nt)
            ds_qrs_dt = create_ds(ds_t, arr_slope_dt)
    ds_qrs_nt.to_netcdf(input_folder_t+'ds_qrs_nt'+folder+'.nc')
    ds_qrs_dt.to_netcdf(input_folder_t+'ds_qrs_dt'+folder+'.nc')

In [10]:
input_folder_t = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA/'
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/'
file_pattern_p = 'PREC_ACC_NC.wrf2d_d01_????-??-??.nc'


folder_names = [
     'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
     'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
     'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
     'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]
for folder in folder_names:
    print(datetime.datetime.now().time())
    full_path_p = os.path.join(base_path, folder, file_pattern_p)
    all_files_p = glob.glob(full_path_p)
    #####
    summer_files_p = [f for f in all_files_p if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_p = xr.open_mfdataset(summer_files_p)
    ds_p = ds_p.sel(time=ds_p['time'].dt.month.isin([6, 7, 8]))
    ds_t = xr.open_mfdataset(input_folder_t+'dn_dewtemp_'+folder+'.nc')
    ds_p_filtered = ds_p.where(ds_p['p'] > 0.1, np.nan)
    
    # 保留特定时间范围内的数据，其他时间标记为nan
    ds_p_daytime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 6) & (ds_p_filtered['time.hour'] < 18), np.nan)
    # 保留18点到次日早上6点的数据，其他时间标记为nan
    ds_p_nighttime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 18) | (ds_p_filtered['time.hour'] < 6), np.nan)
    
    
    arr_dtp = ds_p_daytime.p.values
    arr_ntp = ds_p_nighttime.p.values
    arr_t = ds_t.dnt.values
    
    arr_dt = np.where(np.isnan(arr_dtp), np.nan, arr_t)
    arr_nt = np.where(np.isnan(arr_ntp), np.nan, arr_t)
    
    arr_slope_nt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    arr_slope_dt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    # 循环遍历每个网格点
    
    for i in range(arr_nt.shape[1]):
        for j in range(arr_nt.shape[2]):
            temperature_data_nt = arr_nt[:, i, j]
            precipitation_data_nt = arr_ntp[:, i, j]
            
            temperature_data_dt = arr_dt[:, i, j]
            precipitation_data_dt = arr_dtp[:, i, j]
            # 调用函数并获取斜率
            slope_nt = calculate_ccscale_slope(temperature_data_nt, precipitation_data_nt)[2]
            slope_dt = calculate_ccscale_slope(temperature_data_dt, precipitation_data_dt)[2]
            # 将斜率值存储到arr_slope的对应位置
            arr_slope_nt[i, j] = slope_nt
            arr_slope_dt[i, j] = slope_dt
            ds_lrs_nt = create_ds(ds_t, arr_slope_nt)
            ds_lrs_dt = create_ds(ds_t, arr_slope_dt)
    ds_lrs_nt.to_netcdf(input_folder_t+'ds_dewlrs_nt'+folder+'.nc')
    ds_lrs_dt.to_netcdf(input_folder_t+'ds_dewlrs_dt'+folder+'.nc')

05:57:15.094584
06:03:33.682774
06:12:25.275643
06:22:35.256119
06:37:30.686356
06:56:00.628093
07:16:19.828358
07:37:11.655964
07:52:56.889965
08:02:38.629028
08:09:06.220260
08:20:59.115788
08:36:55.418052
08:54:59.452818
09:15:45.265769
09:40:32.789305
10:06:44.750987
10:33:19.651644
10:41:30.941251
10:48:25.688287
10:57:47.978804
11:14:54.765356
11:34:31.567723
11:55:20.939553
12:15:49.298233
12:35:42.365914
12:56:58.761611
13:08:34.812361
13:14:23.687533
13:19:42.949572
13:25:12.175939
13:32:21.981646
13:39:55.170841
13:49:36.520588
14:00:49.387082
14:13:25.200017


In [10]:
folder_names = [
     'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
     'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
     'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
     'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88',
]
for folder in folder_names:
    print(datetime.datetime.now().time())
    full_path_p = os.path.join(base_path, folder, file_pattern_p)
    all_files_p = glob.glob(full_path_p)
    #####
    summer_files_p = [f for f in all_files_p if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_p = xr.open_mfdataset(summer_files_p)
    ds_p = ds_p.sel(time=ds_p['time'].dt.month.isin([6, 7, 8]))
    ds_t = xr.open_mfdataset(input_folder_t+'dn_dewtemp_'+folder+'.nc')
    ds_p_filtered = ds_p.where(ds_p['p'] > 0.1, np.nan)
    
    # 保留特定时间范围内的数据，其他时间标记为nan
    ds_p_daytime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 6) & (ds_p_filtered['time.hour'] < 18), np.nan)
    # 保留18点到次日早上6点的数据，其他时间标记为nan
    ds_p_nighttime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 18) | (ds_p_filtered['time.hour'] < 6), np.nan)
    
    
    arr_dtp = ds_p_daytime.p.values
    arr_ntp = ds_p_nighttime.p.values
    arr_t = ds_t.dnt.values
    
    arr_dt = np.where(np.isnan(arr_dtp), np.nan, arr_t)
    arr_nt = np.where(np.isnan(arr_ntp), np.nan, arr_t)
    
    arr_slope_nt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    arr_slope_dt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    # 循环遍历每个网格点
    
    for i in range(arr_nt.shape[1]):
        for j in range(arr_nt.shape[2]):
            temperature_data_nt = arr_nt[:, i, j]
            precipitation_data_nt = arr_ntp[:, i, j]
            
            temperature_data_dt = arr_dt[:, i, j]
            precipitation_data_dt = arr_dtp[:, i, j]
            # 调用函数并获取斜率
    
            slope_nt = calculate_ccscale_quantreg(temperature_data_nt, precipitation_data_nt)
            slope_dt = calculate_ccscale_quantreg(temperature_data_dt, precipitation_data_dt)
            # 将斜率值存储到arr_slope的对应位置
            arr_slope_nt[i, j] = slope_nt
            arr_slope_dt[i, j] = slope_dt
            ds_qrs_nt = create_ds(ds_t, arr_slope_nt)
            ds_qrs_dt = create_ds(ds_t, arr_slope_dt)
    ds_qrs_nt.to_netcdf(input_folder_t+'ds_dewqrs_nt'+folder+'.nc')
    ds_qrs_dt.to_netcdf(input_folder_t+'ds_dewqrs_dt'+folder+'.nc')

16:27:40.159428


/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/

16:33:29.675604


/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/

16:43:20.688583


...

/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/

01:00:14.741337


/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/

01:15:44.487503


/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/N/u/gongg/Quartz/.local/lib/python3.11/

In [4]:
input_folder_t = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA/'
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/UTC/'
file_pattern_p = 'PREC_ACC_NC.wrf2d_d01_????-??-??.nc'
output = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA_dailydata/'


folder_names = [
     # 'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
     # 'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
     # 'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
     # 'U-80', 'U-81','U-82', 'U-83', 'U-84', 'U-85', 'U-86', 
    'U-87',# 'U-88',
]
for folder in folder_names:
    print(datetime.datetime.now().time())
    full_path_p = os.path.join(base_path, folder, file_pattern_p)
    all_files_p = glob.glob(full_path_p)
    #####
    summer_files_p = [f for f in all_files_p if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_p = xr.open_mfdataset(summer_files_p)
    ds_p = ds_p.sel(time=ds_p['time'].dt.month.isin([6, 7, 8]))
    ds_t = xr.open_mfdataset(input_folder_t+'dn_temp_'+folder+'.nc')
    ds_dt = xr.open_mfdataset(input_folder_t+'dn_dewtemp_'+folder+'.nc')
    
    
    grouped_p = ds_p.groupby('time.year').groups
    grouped_t = ds_t.groupby('time.year').groups
    grouped_dt = ds_dt.groupby('time.year').groups

    for year, year_indices_p in grouped_p.items():
        year_indices_t = grouped_t[year]
        year_indices_dt = grouped_dt[year]

        ds_year_p = ds_p.isel(time=year_indices_p)
        ds_year_t = ds_t.isel(time=year_indices_t)
        ds_year_dt = ds_dt.isel(time=year_indices_dt)

        monthly_groups_p = ds_year_p.groupby('time.month').groups
        monthly_groups_t = ds_year_t.groupby('time.month').groups
        monthly_groups_dt = ds_year_dt.groupby('time.month').groups

        for month, month_indices_p in monthly_groups_p.items():
            month_indices_t = monthly_groups_t[month]
            month_indices_dt = monthly_groups_dt[month]

            ds_month_p = ds_year_p.isel(time=month_indices_p)
            ds_month_t = ds_year_t.isel(time=month_indices_t)
            ds_month_dt = ds_year_dt.isel(time=month_indices_dt)

            daily_groups_p = ds_month_p.groupby('time.day').groups
            daily_groups_t = ds_month_t.groupby('time.day').groups
            daily_groups_dt = ds_month_dt.groupby('time.day').groups

            # 循环遍历每日的数据
            for day, day_indices_p in daily_groups_p.items():
                day_indices_t = daily_groups_t[day]
                day_indices_dt = daily_groups_dt[day]

                ds_day_p = ds_month_p.isel(time=day_indices_p)
                ds_day_t = ds_month_t.isel(time=day_indices_t)
                ds_day_dt = ds_month_dt.isel(time=day_indices_dt)

                filename_p = f'{folder}_prec_{year}_{month:02d}_{day:02d}.nc'
                filename_t = f'{folder}_dnt_{year}_{month:02d}_{day:02d}.nc'
                filename_dt = f'{folder}_dndt_{year}_{month:02d}_{day:02d}.nc'

                # 导出每日数据为 NetCDF 文件
                ds_day_p.to_netcdf(output+filename_p)
                ds_day_t.to_netcdf(output+filename_t)
                ds_day_dt.to_netcdf(output+filename_dt)

13:41:17.985607


In [2]:
gdf = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
US = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA_dailydata'
output1 = '/N/project/Zli_lab/gongg/CONUS404_data/LST/'
ds_raster = xr.open_dataset('/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/PREC_ACC_NC.wrf2d_d01_2022-09-30_23:00:00.nc')

### classify to 7 region

In [3]:


# 定义所有的U*前缀
prefixes = [
    'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
    'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
    'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
    'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88'
]

for year in range(2014, 2023):  # 从1980年到2022年
    for month in [6, 7, 8]:  # 只读取6, 7, 8月的数据
        print(datetime.datetime.now().time())
        days_in_month = 30 if month == 6 else 31  # 6月30天，7月和8月31天
        
        
        for day in range(1, days_in_month + 1):
            files_to_open = []
            # 对每一个前缀和日期组合构造文件路径
            for prefix in prefixes:
                file_pattern = f'{base_path}/{prefix}_prec_{year}_{month:02d}_{day:02d}.nc'
                files_to_open.append(file_pattern)
            
            ds = xr.open_mfdataset(files_to_open)
            lon = ds_raster['XLONG'].values
            lat = ds_raster['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')

            mask = np.full(ds_raster['PREC_ACC_NC'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds_raster['PREC_ACC_NC'].dims[1:], coords={'south_north': ds_raster['PREC_ACC_NC'].coords['south_north'], 'west_east': ds_raster['PREC_ACC_NC'].coords['west_east']})
            ds_conus = ds_raster.where(mask_da, drop=True)

            XLON = ds_conus.XLONG.values[:707,:]
            XLAT = ds_conus.XLAT.values[:707,:]
            ds_n = ds.assign_coords({
                'XLON': (('lat', 'lon'), XLON),
                'XLAT': (('lat', 'lon'), XLAT)
            })

            regions_dict = {
                'NE': ['CT', 'DE', 'ME', 'MD', 'MA', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT', 'WV'],
                'Midwest': ['IA', 'MI', 'MN', 'WI', 'IL', 'IN', 'MO', 'OH'],
                'SE': ['AL', 'FL', 'GA', 'NC', 'SC', 'VA', 'TN', 'KY', 'AR', 'LA', 'MS'],
                'NGP': ['MT', 'NE', 'ND', 'SD', 'WY'],
                'SGP': ['KS', 'OK', 'TX'],
                'SW': ['AZ', 'CO', 'NM', 'UT', 'CA', 'NV'],
                'NW': ['ID', 'OR', 'WA']
            }
            regions = {name: US[US['STUSPS'].isin(states)] for name, states in regions_dict.items()}
            regi = ['NE','Midwest','SE','NGP','SGP','SW','NW',]

            ds_results = {}

            for r in regi:
                lon = ds_n['XLON'].values
                lat = ds_n['XLAT'].values
                grid = gpd.GeoDataFrame(
                    geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                    index=np.arange(lon.size)
                )

                grid.set_crs(regions[r].crs, inplace=True)
                grid_s = gpd.sjoin(grid, regions[r], how='inner', predicate='within')

                mask = np.full((ds_n['p'].shape[1], ds_n['p'].shape[2]), False) 
                for index in grid_s.index:
                    row, col = np.unravel_index(index, mask.shape)
                    mask[row, col] = True 

                mask_da = xr.DataArray(
                    mask, 
                    dims=['lat', 'lon'],
                    coords={
                        'lat': ds_n['lat'].values,
                        'lon': ds_n['lon'].values
                    }
                )

                ds_ssss = ds_n.where(mask_da, drop=True)
                ds_results[f'ds_{r}'] = ds_ssss

            ds_results['ds_NE'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}NE/prec_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_Midwest'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}Midwest/prec_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_SE'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}SE/prec_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_NGP'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}NGP/prec_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_SGP'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}SGP/prec_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_SW'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}SW/prec_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_NW'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}NW/prec_{year}_{month:02d}_{day:02d}.nc")


19:30:39.823650
19:37:39.649584
19:44:39.761013
19:52:43.280911
20:04:03.119372
20:15:59.551287
20:27:54.821219
20:41:10.825572
20:54:33.369666
21:09:19.084338
21:23:05.104875
21:38:24.729327
21:52:20.588817
22:06:30.184047
22:21:13.539447
22:34:29.795321
22:47:51.864598
23:01:09.614417
23:14:08.866354
23:26:49.009688
23:39:35.607301
23:51:58.202999
00:04:19.770188
00:17:31.996993
00:30:04.459286
00:42:43.271731
00:57:05.073107


###  CC 7 region

In [3]:
ds_p = xr.open_mfdataset('/N/project/Zli_lab/gongg/CONUS404_data/LST/SGP/prec*.nc')

In [5]:
arr_p = ds_p.p.values

In [7]:
arr_p.shape

(94944, 393, 308)

In [ ]:
ds_t = xr.open_mfdataset('/N/project/Zli_lab/gongg/CONUS404_data/LST/SGP/dnt*.nc')

In [ ]:
folder_names = [ 'NE', ]

for folder in folder_names:
    print(datetime.datetime.now().time())
    full_path_p = os.path.join(base_path, folder, file_pattern_p)
    all_files_p = glob.glob(full_path_p)
    #####
    summer_files_p = [f for f in all_files_p if '-06-' in f or '-07-' in f or '-08-' in f or '-09-' in f]
    ds_p = xr.open_mfdataset(summer_files_p)
    ds_p = ds_p.sel(time=ds_p['time'].dt.month.isin([6, 7, 8]))
    ds_t = xr.open_mfdataset(input_folder_t+'dn_dewtemp_'+folder+'.nc')
    ds_p_filtered = ds_p.where(ds_p['p'] > 0.1, np.nan)
    
    # 保留特定时间范围内的数据，其他时间标记为nan
    ds_p_daytime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 6) & (ds_p_filtered['time.hour'] < 18), np.nan)
    # 保留18点到次日早上6点的数据，其他时间标记为nan
    ds_p_nighttime = ds_p_filtered.where((ds_p_filtered['time.hour'] >= 18) | (ds_p_filtered['time.hour'] < 6), np.nan)
    
    
    arr_dtp = ds_p_daytime.p.values
    arr_ntp = ds_p_nighttime.p.values
    arr_t = ds_t.dnt.values
    
    arr_dt = np.where(np.isnan(arr_dtp), np.nan, arr_t)
    arr_nt = np.where(np.isnan(arr_ntp), np.nan, arr_t)
    
    arr_slope_nt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    arr_slope_dt = np.full((arr_nt.shape[1], arr_nt.shape[2]), np.nan)
    # 循环遍历每个网格点
    
    for i in range(arr_nt.shape[1]):
        for j in range(arr_nt.shape[2]):
            temperature_data_nt = arr_nt[:, i, j]
            precipitation_data_nt = arr_ntp[:, i, j]
            
            temperature_data_dt = arr_dt[:, i, j]
            precipitation_data_dt = arr_dtp[:, i, j]
            # 调用函数并获取斜率
    
            slope_nt = calculate_ccscale_quantreg(temperature_data_nt, precipitation_data_nt)
            slope_dt = calculate_ccscale_quantreg(temperature_data_dt, precipitation_data_dt)
            # 将斜率值存储到arr_slope的对应位置
            arr_slope_nt[i, j] = slope_nt
            arr_slope_dt[i, j] = slope_dt
            ds_qrs_nt = create_ds(ds_t, arr_slope_nt)
            ds_qrs_dt = create_ds(ds_t, arr_slope_dt)
    ds_qrs_nt.to_netcdf(input_folder_t+'ds_dewqrs_nt'+folder+'.nc')
    ds_qrs_dt.to_netcdf(input_folder_t+'ds_dewqrs_dt'+folder+'.nc')

In [ ]:
gdf = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
US = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA_dailydata'
output1 = '/N/project/Zli_lab/gongg/CONUS404_data/LST/'
ds_raster = xr.open_dataset('/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/PREC_ACC_NC.wrf2d_d01_2022-09-30_23:00:00.nc')
# 定义所有的U*前缀
prefixes = [
    'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
    'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
    'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
    'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88'
]

for year in range(1990, 2000):  # 从1980年到2022年
    for month in [6, 7, 8]:  # 只读取6, 7, 8月的数据
        print(datetime.datetime.now().time())
        days_in_month = 30 if month == 6 else 31  # 6月30天，7月和8月31天
        
        
        for day in range(1, days_in_month + 1):
            files_to_open = []
            # 对每一个前缀和日期组合构造文件路径
            for prefix in prefixes:
                file_pattern = f'{base_path}/{prefix}_mdt_{year}_{month:02d}_{day:02d}.nc'
                files_to_open.append(file_pattern)
                
            ds = xr.open_mfdataset(files_to_open)
            lon = ds_raster['XLONG'].values
            lat = ds_raster['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')

            mask = np.full(ds_raster['PREC_ACC_NC'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds_raster['PREC_ACC_NC'].dims[1:], coords={'south_north': ds_raster['PREC_ACC_NC'].coords['south_north'], 'west_east': ds_raster['PREC_ACC_NC'].coords['west_east']})
            ds_conus = ds_raster.where(mask_da, drop=True)

            XLON = ds_conus.XLONG.values[:707,:]
            XLAT = ds_conus.XLAT.values[:707,:]
            ds_n = ds.assign_coords({
                'XLON': (('lat', 'lon'), XLON),
                'XLAT': (('lat', 'lon'), XLAT)
            })

            regions_dict = {
                'NE': ['CT', 'DE', 'ME', 'MD', 'MA', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT', 'WV'],
                'Midwest': ['IA', 'MI', 'MN', 'WI', 'IL', 'IN', 'MO', 'OH'],
                'SE': ['AL', 'FL', 'GA', 'NC', 'SC', 'VA', 'TN', 'KY', 'AR', 'LA', 'MS'],
                'NGP': ['MT', 'NE', 'ND', 'SD', 'WY'],
                'SGP': ['KS', 'OK', 'TX'],
                'SW': ['AZ', 'CO', 'NM', 'UT', 'CA', 'NV'],
                'NW': ['ID', 'OR', 'WA']
            }
            regions = {name: US[US['STUSPS'].isin(states)] for name, states in regions_dict.items()}
            regi = ['NE','Midwest','SE','NGP','SGP','SW','NW',]

            ds_results = {}

            for r in regi:
                lon = ds_n['XLON'].values
                lat = ds_n['XLAT'].values
                grid = gpd.GeoDataFrame(
                    geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                    index=np.arange(lon.size)
                )

                grid.set_crs(regions[r].crs, inplace=True)
                grid_s = gpd.sjoin(grid, regions[r], how='inner', predicate='within')

                mask = np.full((ds_n['td2'].shape[1], ds_n['td2'].shape[2]), False) 
                for index in grid_s.index:
                    row, col = np.unravel_index(index, mask.shape)
                    mask[row, col] = True 

                mask_da = xr.DataArray(
                    mask, 
                    dims=['lat', 'lon'],
                    coords={
                        'lat': ds_n['lat'].values,
                        'lon': ds_n['lon'].values
                    }
                )

                ds_ssss = ds_n.where(mask_da, drop=True)
                ds_results[f'ds_{r}'] = ds_ssss

            ds_results['ds_NE'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}NE/mdt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_Midwest'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}Midwest/mdt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_SE'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}SE/mdt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_NGP'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}NGP/mdt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_SGP'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}SGP/mdt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_SW'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}SW/mdt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_NW'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}NW/mdt_{year}_{month:02d}_{day:02d}.nc")

16:19:46.446123
16:30:02.195981
16:40:54.843250
16:53:04.847060


In [2]:
gdf = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
US = gpd.read_file('/N/project/Zli_lab/Data/Other/tl_2019_us_state/tl_2019_us_state.shp')
base_path = '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA_dailydata'
output1 = '/N/project/Zli_lab/gongg/CONUS404_data/LST/'
ds_raster = xr.open_dataset('/N/project/Zli_lab/Data/Observations/NCAR/prec_acc_files/PREC_ACC_NC.wrf2d_d01_2022-09-30_23:00:00.nc')
# 定义所有的U*前缀
prefixes = [
    'U-50', 'U-51', 'U-52', 'U-53', 'U-54', 'U-55', 'U-56', 'U-57', 'U-58',
    'U-60', 'U-61', 'U-62', 'U-63', 'U-64', 'U-65', 'U-66', 'U-67', 'U-68',
    'U-70', 'U-71', 'U-72', 'U-73', 'U-74', 'U-75', 'U-76', 'U-77', 'U-78',
    'U-80', 'U-81', 'U-82', 'U-83', 'U-84', 'U-85', 'U-86', 'U-87', 'U-88'
]

for year in range(1990, 2001):  # 从1980年到2022年
    for month in [6, 7, 8]:  # 只读取6, 7, 8月的数据
        print(datetime.datetime.now().time())
        days_in_month = 30 if month == 6 else 31  # 6月30天，7月和8月31天
        
        
        for day in range(1, days_in_month + 1):
            files_to_open = []
            # 对每一个前缀和日期组合构造文件路径
            for prefix in prefixes:
                file_pattern = f'{base_path}/{prefix}_mt_{year}_{month:02d}_{day:02d}.nc'
                files_to_open.append(file_pattern)
                
            ds = xr.open_mfdataset(files_to_open)
            lon = ds_raster['XLONG'].values
            lat = ds_raster['XLAT'].values
            grid = gpd.GeoDataFrame(
                geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                index=np.arange(lon.size)
            )
            grid.set_crs(gdf.crs, inplace=True)
            grid_s = gpd.sjoin(grid, gdf, how='inner', predicate='within')

            mask = np.full(ds_raster['PREC_ACC_NC'].shape[1:], False) 
            for index in grid_s.index:
                row, col = np.unravel_index(index, mask.shape)  # 获取行列索引
                mask[row, col] = True
            mask_da = xr.DataArray(mask, dims=ds_raster['PREC_ACC_NC'].dims[1:], coords={'south_north': ds_raster['PREC_ACC_NC'].coords['south_north'], 'west_east': ds_raster['PREC_ACC_NC'].coords['west_east']})
            ds_conus = ds_raster.where(mask_da, drop=True)

            XLON = ds_conus.XLONG.values[:707,:]
            XLAT = ds_conus.XLAT.values[:707,:]
            ds_n = ds.assign_coords({
                'XLON': (('lat', 'lon'), XLON),
                'XLAT': (('lat', 'lon'), XLAT)
            })

            regions_dict = {
                'NE': ['CT', 'DE', 'ME', 'MD', 'MA', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT', 'WV'],
                'Midwest': ['IA', 'MI', 'MN', 'WI', 'IL', 'IN', 'MO', 'OH'],
                'SE': ['AL', 'FL', 'GA', 'NC', 'SC', 'VA', 'TN', 'KY', 'AR', 'LA', 'MS'],
                'NGP': ['MT', 'NE', 'ND', 'SD', 'WY'],
                'SGP': ['KS', 'OK', 'TX'],
                'SW': ['AZ', 'CO', 'NM', 'UT', 'CA', 'NV'],
                'NW': ['ID', 'OR', 'WA']
            }
            regions = {name: US[US['STUSPS'].isin(states)] for name, states in regions_dict.items()}
            regi = ['NE','Midwest','SE','NGP','SGP','SW','NW',]

            ds_results = {}

            for r in regi:
                lon = ds_n['XLON'].values
                lat = ds_n['XLAT'].values
                grid = gpd.GeoDataFrame(
                    geometry=gpd.points_from_xy(lon.flatten(), lat.flatten()),
                    index=np.arange(lon.size)
                )

                grid.set_crs(regions[r].crs, inplace=True)
                grid_s = gpd.sjoin(grid, regions[r], how='inner', predicate='within')

                mask = np.full((ds_n['t2'].shape[1], ds_n['t2'].shape[2]), False) 
                for index in grid_s.index:
                    row, col = np.unravel_index(index, mask.shape)
                    mask[row, col] = True 

                mask_da = xr.DataArray(
                    mask, 
                    dims=['lat', 'lon'],
                    coords={
                        'lat': ds_n['lat'].values,
                        'lon': ds_n['lon'].values
                    }
                )

                ds_ssss = ds_n.where(mask_da, drop=True)
                ds_results[f'ds_{r}'] = ds_ssss

            ds_results['ds_NE'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}NE/mt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_Midwest'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}Midwest/mt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_SE'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}SE/mt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_NGP'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}NGP/mt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_SGP'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}SGP/mt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_SW'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}SW/mt_{year}_{month:02d}_{day:02d}.nc")
            ds_results['ds_NW'].drop_vars(['XLON', 'XLAT']).to_netcdf(f"{output1}NW/mt_{year}_{month:02d}_{day:02d}.nc")

17:43:00.397521


FileNotFoundError: [Errno 2] No such file or directory: '/N/project/Zli_lab/gongg/CONUS404_data/LST/JJA_dailydata/U-58_mt_1990_06_01.nc'